In [11]:
## Polecenia do instalacji pakietów
# conda install -c conda-forge tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
data_AUG = pd.read_csv("apartments_pl_2023_08.csv")
data_SEP = pd.read_csv("apartments_pl_2023_09.csv")
data_OCT = pd.read_csv("apartments_pl_2023_10.csv")

In [6]:
len(data_AUG)

18905

In [7]:
len(data_SEP)

16997

In [8]:
len(data_OCT)

16690